# Genetic Load in non-equilibrium

(c) 2020 Tom Röschinger. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

In [1]:
using LambertW, Jedi, Plots, CSV, DataFrames, Statistics, Measures
Jedi.default_plotlyjs!()

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-4332745716790012358\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-3846333422537543740\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-4332745716790012358\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-17047902474337662476\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-15405514411786487728\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-17047902474337662476\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /Users/tomroschinger/.julia/packages/ORCA/U5XaN/src/ORCA.jl:8


Plots.PlotlyJSBackend()

Here we are going to look at the different components of the genetic load and find the length which minimizes the load for any given ratio of the mutation rates.

In [2]:
function theoretical_load(rho, F, fl)
    Load = zeros(23)
    F.fl = fl
    for (j, l_0) in enumerate(8:30)
        Q_0_smooth = zeros(Float64, (l_0+1)*10)
        for (i, l) in enumerate(0:0.1:l_0)
            Q_0_smooth[i] = exp(-1/2*(l-l_0/4*3)^2/(3*l_0/16)) * exp(2N/(1+rho)*fitness(2l, l_0, F))
        end
        Z = lambertw(3/4 * N * F.f0 * l_0 * exp(10)/(1+rho))
        

        Load[j] = 1/(N * l_0) * Z * (1+rho) + F.fl * l_0
    end
    lopt = sqrt(lambertw(3/4 * N * F.f0 * 10 * exp(10)/(1 + rho))/(N * F.fl) *(1+rho))
    return Load, lopt
end

theoretical_load (generic function with 1 method)

In [3]:
N = 1000
f0 = 50/2N
fl = 0.3/2N
F = Jedi.fermi_fitness(f0=f0, fl=fl)

fermi_fitness(10, 1.0, 0.025, 0.00015, Jedi.Est)

Test selection coefficients.

In [4]:
f_p = Jedi.dE_fitness.(collect(0:0.01:1).*20, 10, F)

plot(collect(0:0.01:1).*20, 2N*abs.(f_p), label=:none)

<!DOCTYPE html>
 
 
 Plots.jl

First we check out the load in equilibrium. We need to do this to find the right parameter that gives the fitness penalty for length. We choose this parameter such that for $\rho=0$, the optimal length is about $l=10$. In the plot 

In [5]:
plot(8:30, theoretical_load(0, F, 0)[1], linewidth=2, label="Load_k")
plot!(8:30, [fl* l for l in 8:30], linewidth=2, label="Load_l")
plot!(8:30, theoretical_load(0, F, fl)[1], linewidth=2, label="total load")

<!DOCTYPE html>
 
 
 Plots.jl

In [6]:
plot(8:30, theoretical_load(2, F, 0)[1], linewidth=2, label="Load_k")
plot!(8:30, [fl * l for l in 8:30], linewidth=2, label="Load_l")
plot!(8:30, theoretical_load(2, F, fl)[1], linewidth=2, label="total load")

<!DOCTYPE html>
 
 
 Plots.jl

In [7]:
l_opt = []
x = []
for rho in 0.01:0.01:2
    L, l = theoretical_load(rho, F, fl)
    push!(l_opt, argmin(L))
    push!(x, l)
end
plot(0.01:0.01:2, l_opt.+7)
plot!(0.01:0.01:2, x)


<!DOCTYPE html>
 
 
 Plots.jl

In [8]:
plot(
    0.01:0.01:2, 
    x,
    xlabel="ρ",
    ylabel="l_opt",
    linewidth=2,
    left_margin=3mm
)

<!DOCTYPE html>
 
 
 Plots.jl

In [9]:
savefig("../figures/optimal_length_from_load.pdf")

Data

In [10]:
df = CSV.read("../outputs/2020_08_27_script2_results.csv")
gdf = groupby(df, :rho)
cdf = combine(:l => (x) -> (mean=mean(x), variance=var(x)), gdf)

,rho,mean,variance
,Float64,Float64,Float64
1,0.0,15.087,20.2577
2,0.1,14.83,26.0191
3,0.5,10.936,45.3072
4,1.0,8.42,34.3399
5,2.0,7.214,27.5838
6,4.0,6.918,22.4577
